In [ ]:
import tensorflow as tf
import cv2
from matplotlib import pyplot as plt
import os
from scipy import misc
import numpy as np
from model import build_model, get_loss, get_train_op

In [ ]:
num_classes = 40
input_image = tf.placeholder(shape=(None, None, None, 3), dtype=tf.float32)
input_labels = tf.placeholder(shape=(None, None, None, 1), dtype=tf.float32)
learning_rate = tf.placeholder(shape=(), dtype=tf.float32)

model_out, resized_model_out = build_model(input_image, num_classes)
model_loss = get_loss(model_out, input_labels)

train_op = get_train_op(model_loss, learning_rate)

print('Model Output : ', model_out)
print('Postprocessed Output : ', resized_model_out)

In [ ]:
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     writer = tf.summary.FileWriter('./logs/', sess.graph)
#     writer.close()

In [ ]:
def extract_fn(data_tfrecord):
    features = {
        'image' : tf.FixedLenFeature([], tf.string),
        'mask' : tf.FixedLenFeature([], tf.string)
    }
    sample = tf.parse_single_example(data_tfrecord, features)
    image = tf.image.decode_image(sample['image'])
    mask = tf.image.decode_image(sample['mask'])
    
    return image, mask

In [ ]:
training_filenames = ['./../preperation/IDD/TFRecords/' + ele for ele in os.listdir('./../preperation/IDD/TFRecords/')]
filenames = tf.placeholder(tf.string, shape=[None])
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(extract_fn)
dataset = dataset.repeat()
dataset = dataset.batch(4)
# iterator = dataset.make_one_shot_iterator()
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10):
        while True:
            sess.run(iterator.initializer, feed_dict={filenames: training_filenames})
            data_record = sess.run(next_element)
            _, loss = sess.run([train_op, model_loss], feed_dict={input_image:data_record[0], input_labels:np.expand_dims(data_record[1][:, :, :, 0], axis=-1), learning_rate:1e-3})
            print(loss)
    print('Epoch Completed')
#         data_record = sess.run(next_element)
#         print(type(data_record))
#         new_im = cv2.cvtColor(data_record[1], cv2.COLOR_BGR2RGB)
#         cv2.imwrite('a.jpg', new_im)
#         cv2.imshow('Image', new_im)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()